In [3]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import os
import json
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

import pandas as pd
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import TfidfVectorizer


ruta = "F:/nuevo_conocimiento/CODIGO DE TESIS (MUY IMPORTANTE)/Experimentos_con_modelos_basicos/dataset_preprocesado/dataset_abstract_preprocesado.csv"

datos = pd.read_csv(ruta)
datos

,title,abstract,categorical,categoria,cat_one_hot,abstract_limpio,abstract_preprocesado
0,Electromagnetic neutrinos in laboratory experi...,An overview of neutrino electromagnetic proper...,astrophysics,humano,"[1, 0, 0, 0, 0]",An overview of neutrino electromagnetic proper...,"['overview', 'neutrino', 'electromagnetic', 'p..."
1,Infrared complex refractive index of astrophys...,In dense and cold regions of the interstellar ...,astrophysics,humano,"[1, 0, 0, 0, 0]",In dense and cold regions of the interstellar ...,"['dense', 'cold', 'region', 'interstellar', 'm..."
2,Gamma-ray lines in modern astrophysics,Gamma-ray astronomy provides a direct window i...,astrophysics,humano,"[1, 0, 0, 0, 0]",Gamma-ray astronomy provides a direct window i...,"['gamma-ray', 'astronomy', 'provides', 'direct..."
3,From Coulomb excitation cross sections to non-...,Coulomb and nuclear dissociation of $^{17}$Ne ...,astrophysics,humano,"[1, 0, 0, 0, 0]",Coulomb and nuclear dissociation of $^{17}$Ne ...,"['coulomb', 'nuclear', 'dissociation', 'ne', '..."
4,Astrophysical Black Holes: A Compact Pedagogic...,Black holes are among the most extreme objects...,astrophysics,humano,"[1, 0, 0, 0, 0]",Black holes are among the most extreme objects...,"['black', 'hole', 'among', 'extreme', 'object'..."
...,...,...,...,...,...,...,...
7745,Superconducting LaAlO3/SrTiO3 Nanowires,Here's a rewritten version of the text using m...,superconductivity,llama3,"[0, 0, 0, 0, 1]",\n\nWe've achieved a groundbreaking breakthrou...,"[""'ve"", 'achieved', 'groundbreaking', 'breakth..."
7746,Coupling a single Nitrogen-Vacancy center to a...,Here's a rewritten version of the text in my o...,superconductivity,llama3,"[0, 0, 0, 0, 1]",\n\nWe propose a groundbreaking idea: connecti...,"['propose', 'groundbreaking', 'idea', 'connect..."
7747,Superconducting and Anti-Ferromagnetic Phases ...,Here's a rewritten version of the text using m...,superconductivity,llama3,"[0, 0, 0, 0, 1]",\n\nThis study establishes a connection betwee...,"['study', 'establishes', 'connection', 'two', ..."
7748,New Superconductivity Dome in LaFeAsO$_{1-x}$F...,Here's the rewritten text:\n\nA fascinating ph...,superconductivity,llama3,"[0, 0, 0, 0, 1]",Here's the rewritten text:\n\nA fascinating ph...,"[""'s"", 'rewritten', 'text', 'fascinating', 'ph..."


In [4]:
from llama_index.embeddings.ollama import OllamaEmbedding

ollama_embedding = OllamaEmbedding(
    model_name="llama2:7b",
    base_url="http://localhost:11434",
    ollama_additional_kwargs={"mirostat": 0},
)

In [5]:
textos = datos['abstract_limpio'].tolist()
etiquetas = datos['cat_one_hot'].tolist()

pass_embedding = ollama_embedding.get_text_embedding_batch(
   textos, show_progress=True
)

print(len(pass_embedding))

Generating embeddings:   0%|          | 0/7750 [00:00<?, ?it/s]

7750


In [6]:
df = pd.DataFrame(pass_embedding)
df

,0,1,2,3,4,5,6,7,8,9,...,4086,4087,4088,4089,4090,4091,4092,4093,4094,4095
0,1.188418,-1.736702,-0.191322,-1.994687,0.057326,2.883176,-0.838245,0.477511,2.910899,3.599902,...,-0.379370,-1.576188,-2.516441,0.888087,0.380534,-0.241870,1.902041,-1.295173,-0.481783,0.461955
1,1.457455,0.817127,1.458703,-1.812698,0.367964,1.609878,-1.075397,-1.194629,-0.962105,1.071324,...,-0.733655,-0.815370,0.260745,-0.913560,1.406413,-1.041576,0.792303,0.447136,0.357298,-0.323983
2,0.149753,-1.643355,0.711284,-2.487119,-2.410948,-0.555468,-0.380990,-2.519633,1.750998,1.200407,...,0.761094,-1.961351,1.976893,-0.154175,1.196034,0.968694,0.444169,1.756454,-0.673243,0.327200
3,1.069569,-0.899952,1.030322,-1.479048,-0.635635,0.216762,0.876879,-0.637044,1.633442,3.777382,...,-1.102170,-1.118121,2.991307,0.879041,-0.939751,-2.201182,1.924992,-0.070183,1.336386,-1.397303
4,0.505189,-1.607398,2.388520,-2.933331,-0.703085,1.615509,0.834435,-0.337594,-0.439151,1.605850,...,0.241560,-1.599698,-0.193177,0.144442,0.711942,-1.273402,2.305234,0.856522,1.108208,0.818920
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7745,2.740937,0.568207,3.251906,0.036487,-0.869197,0.512776,0.136915,-1.405676,-1.050151,2.209044,...,-0.667552,1.072307,-0.598831,-0.979040,-0.541928,-0.016980,0.635663,0.578578,-0.159424,-1.876918
7746,-0.103337,-0.998430,2.602769,-1.710451,-2.077935,-0.288074,-0.545128,-1.718456,-1.177045,1.770366,...,-1.024788,-1.388594,0.103467,-0.946912,0.450563,-0.920542,0.565654,0.325072,-0.930094,0.669088
7747,2.312369,-0.996369,3.858977,-0.668349,-1.882364,1.540483,-1.182332,-1.023989,-0.327760,-0.009968,...,0.083419,0.704278,-0.683311,-1.342623,1.571705,-1.704096,1.029309,1.449589,2.328573,-0.148603
7748,2.373055,1.133662,0.727872,-0.413509,-1.462697,1.553015,0.258164,-2.528388,-1.398040,2.430077,...,0.015404,1.957772,-0.375481,0.023945,2.043267,0.078766,-0.393090,-1.101733,-0.256940,0.026896


In [7]:
df['cat_one_hot'] = etiquetas
df.to_csv('llama2_embeddings.csv', index=False)  # Para CSV